In [3]:
import cv2
import mediapipe as mp
import os
from tqdm import tqdm  # progress bar

# Initialize Mediapipe face detector
mp_face_detection = mp.solutions.face_detection
detector = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# Input (raw dataset) and output (cropped faces) directories
input_dir = r"C:\Users\Sandesh Prasai\Downloads\Pictures"
output_dir = r"C:\Users\Sandesh Prasai\Downloads\Pictures\Faces"

os.makedirs(output_dir, exist_ok=True)

# Padding factors
padding_factor = 0.25      # default padding for sides
extra_head_padding = 0.15  # extra padding for head only
reduced_bottom_padding = 0.10  # CHANGED: reduced padding below face (chest area)

# Count total images for progress bar
total_images = sum(
    len(files) for _, _, files in os.walk(input_dir)
)

with tqdm(total=total_images, desc="Processing Images", unit="img") as pbar:
    # Loop through each person's folder
    for person_name in os.listdir(input_dir):
        person_path = os.path.join(input_dir, person_name)
        if not os.path.isdir(person_path):
            continue  # skip if not a folder

        # Create corresponding output folder
        person_output = os.path.join(output_dir, person_name)
        os.makedirs(person_output, exist_ok=True)

        # Loop through each image in the folder
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)

            if img is None:
                pbar.update(1)
                continue

            # Convert to RGB for Mediapipe
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = detector.process(rgb)

            if not results.detections:
                pbar.update(1)
                continue  # skip if no face found

            # Find the largest face
            largest = max(
                results.detections,
                key=lambda det: det.location_data.relative_bounding_box.width *
                                det.location_data.relative_bounding_box.height
            )

            # Extract bounding box
            box = largest.location_data.relative_bounding_box
            h, w, _ = img.shape
            x, y = int(box.xmin * w), int(box.ymin * h)
            bw, bh = int(box.width * w), int(box.height * h)

            # Add padding with different values for each side
            pad_w = int(bw * padding_factor)  # same padding for left/right sides
            pad_top = int(bh * (padding_factor + extra_head_padding))  # extra padding on top (head)
            pad_bottom = int(bh * reduced_bottom_padding)  # CHANGED: reduced padding below (chest)

            x1 = max(0, x - pad_w)
            y1 = max(0, y - pad_top)    # more padding towards head
            x2 = min(w, x + bw + pad_w)
            y2 = min(h, y + bh + pad_bottom)  # CHANGED: less padding below face

            face = img[y1:y2, x1:x2]

            if face.size == 0:
                pbar.update(1)
                continue

            # Save cropped face (no resizing)
            out_path = os.path.join(person_output, img_name)
            cv2.imwrite(out_path, face)

            pbar.update(1)

print("✅ Cropping completed with extra head padding and reduced chest padding! Faces saved in:", output_dir)

Processing Images: 243img [00:39,  6.08img/s]                                                                          

✅ Cropping completed with extra head padding and reduced chest padding! Faces saved in: C:\Users\Sandesh Prasai\Downloads\Pictures\Faces


** Code to delete images that are small than 224 * 224 Size 

In [2]:
import cv2
import os
from tqdm import tqdm  # for progress bar

# Directory containing cropped face images
faces_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\data\faces"

# Minimum width and height
min_size = 224

# Count total images for progress bar
total_images = sum(
    len(files) for _, _, files in os.walk(faces_dir)
)

with tqdm(total=total_images, desc="Checking Images", unit="img") as pbar:
    # Loop through each person's folder
    for person_name in os.listdir(faces_dir):
        person_path = os.path.join(faces_dir, person_name)
        if not os.path.isdir(person_path):
            continue

        # Loop through each image in the folder
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)

            if img is None:
                pbar.update(1)
                continue

            h, w, _ = img.shape

            # Delete image if smaller than min_size
            if h < min_size or w < min_size:
                os.remove(img_path)
                # print(f"🗑️ Deleted {img_path} ({w}x{h})")

            pbar.update(1)

print("✅ Small images deleted!")


Checking Images: 100%|██████████| 16058/16058 [02:17<00:00, 116.60img/s]

✅ Small images deleted!


In [4]:
import os

# Directory containing all person folders
faces_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\data\faces"

min_images =  70 # Threshold for minimum images
folders_below_threshold = 0

print("📂 Image Count per Person")
print("=" * 40)

# Loop through each person's folder
for person_name in os.listdir(faces_dir):
    person_path = os.path.join(faces_dir, person_name)
    if not os.path.isdir(person_path):
        continue

    # Count image files (common image extensions)
    image_count = sum(
        1 for f in os.listdir(person_path)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))
    )

    print(f"{person_name:<20} : {image_count} images")

    # Check if below threshold
    if image_count < min_images:
        folders_below_threshold += 1

print("=" * 40)
print(f"Number of folders with less than {min_images} images: {folders_below_threshold}")


📂 Image Count per Person
Aishwarya_Rai        : 82 images
Akshay_Kumar         : 73 images
Alia_Bhatt           : 138 images
Allu_Arjun           : 60 images
Amitabh_Bachchan     : 95 images
Angelina_Jolie       : 75 images
Ariana_Grande        : 97 images
Barack_Obama         : 162 images
Billie_Eilish        : 79 images
Bill_Gates           : 124 images
Brad_Pitt            : 75 images
Chris_Evans          : 77 images
Cristiano_Ronaldo    : 89 images
Deepika_Padukone     : 77 images
Drake                : 87 images
Dwayne_Johnson       : 86 images
Ed_Sheeran           : 81 images
Emma_Watson          : 96 images
Hrithik_Roshan       : 93 images
Jackie_Chan          : 116 images
Jeff_Bezos           : 75 images
Jennifer_Lawrence    : 98 images
Justin_Bieber        : 88 images
Kamal_Haasan         : 71 images
Kareena_Kapoor       : 63 images
Kevin_Hart           : 73 images
Kim_Kardashian       : 95 images
Kylie_Jenner         : 80 images
Leonardo_DiCaprio    : 82 images
Mark_Zuckerber